# Model Development For Khulna District
In this notebook, we will explore Linear Regression & LSTM modelling for Khulna district. <br>
The target feature in this notebook is 'precip'. <br>
There are 3 dataset namely training, validation & evaluation dataset. <br>
In model evaluation, evaluation metric RMSE is used for hyperparameter iteration. The final model is evaluated based on MAE and R2 Score.

In [100]:
# Import libraires
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import tensorflow as tf

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from itertools import product


from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

In [101]:
# Load dataset
file_path = 'Khulna_Weather_combined.csv'

# Load dataset using absolute file path
df_khulna = pd.read_csv(file_path)

## 1) Data preprocessing

In [102]:
# First 5 rows of df_khulna dataset
df_khulna.head()

,datetime,Unnamed: 0_x,name,tempmax_x,tempmin_x,temp,feelslikemax,feelslikemin,feelslike,dew,...,weathercode,tempmax_y,tempmin_y,temperature_2m_mean,apparent_temperature_mean,precipitation_sum,rain_sum,precipitation_hours,windspeed_y,et0_fao_evapotranspiration
0,01-01-2012,0,"khulna,Bangladesh",31.0,15.7,22.2,29.6,15.7,22.0,12.7,...,51,27.7,17.4,22.1,22.1,0.1,0.1,1.0,11.8,3.57
1,01-01-2015,488,"Khulna,Bangladesh",28.9,17.5,21.2,28.8,17.5,21.2,16.5,...,53,21.7,18.3,20.1,22.0,7.0,7.0,13.0,10.8,0.82
2,01-01-2016,853,"Khulna,Bangladesh",28.0,12.8,19.4,27.5,12.8,19.4,11.8,...,1,26.2,13.8,19.9,20.5,0.0,0.0,0.0,9.0,3.04
3,01-01-2016,2435,"Khulna,Bangladesh",28.0,12.8,19.4,27.5,12.8,19.4,11.8,...,1,26.2,13.8,19.9,20.5,0.0,0.0,0.0,9.0,3.04
4,01-01-2017,974,"Khulna,Bangladesh",26.7,14.5,19.8,26.7,14.5,19.8,14.6,...,3,25.7,15.0,19.8,20.9,0.0,0.0,0.0,9.2,2.90


In [103]:
# Summary of df_khulna dataset
df_khulna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3515 entries, 0 to 3514
Data columns (total 45 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   datetime                    3515 non-null   object 
 1   Unnamed: 0_x                3515 non-null   int64  
 2   name                        3515 non-null   object 
 3   tempmax_x                   3515 non-null   float64
 4   tempmin_x                   3515 non-null   float64
 5   temp                        3515 non-null   float64
 6   feelslikemax                3515 non-null   float64
 7   feelslikemin                3515 non-null   float64
 8   feelslike                   3515 non-null   float64
 9   dew                         3515 non-null   float64
 10  humidity                    3515 non-null   float64
 11  precip                      3515 non-null   float64
 12  precipprob                  3515 non-null   float64
 13  precipcover                 3515 

In [104]:
# Check for duplicates based on datetime
duplicate_rows_subset = df_khulna[df_khulna.duplicated(subset=['datetime'])]

# Display duplicate rows
if not duplicate_rows_subset.empty:
    print("Duplicate rows found based on the subset columns:")
    print(duplicate_rows_subset)
else:
    print("No duplicate rows found based on the subset columns.")

Duplicate rows found based on the subset columns:
        datetime  Unnamed: 0_x               name  tempmax_x  tempmin_x  temp  \
3     01-01-2016          2435  Khulna,Bangladesh       28.0       12.8  19.4   
14    01-02-2016          2466  Khulna,Bangladesh       31.7       18.7  23.5   
25    01-03-2016          2495  Khulna,Bangladesh       33.3       20.4  26.0   
36    01-04-2016          2526  Khulna,Bangladesh       33.2       19.0  26.6   
119   02-01-2016          2436  Khulna,Bangladesh       27.9       12.2  19.1   
...          ...           ...                ...        ...        ...   ...   
3348  30-01-2016          2464  Khulna,Bangladesh       29.6       14.3  21.1   
3359  30-03-2016          2524  Khulna,Bangladesh       36.1       24.8  29.6   
3370  30-04-2016          2555  Khulna,Bangladesh       41.0       27.3  32.9   
3452  31-01-2016          2465  Khulna,Bangladesh       31.1       17.0  22.6   
3463  31-03-2016          2525  Khulna,Bangladesh       34.

In [105]:
# Remove duplicates from the entire DataFrame
df_khulna = df_khulna.drop_duplicates(subset=['datetime'])

In [106]:
# Count the number of rows with missing values in the DataFrame
num_missing_rows = df_khulna.isna().any(axis=1).sum()

# Display the number of rows with missing values
print(f"Number of rows with missing values: {num_missing_rows}")

# Fill missing values with 0 in the original DataFrame
df_khulna.fillna(0, inplace=True)

Number of rows with missing values: 3066


Next, we will drop features that have weak correlation with 'precip' or are not necessary.

In [107]:
# Drop features
features_to_drop = [
    'Unnamed: 0_x', 'name', 'snow',
    'snowdepth', 'windgust', 'severerisk',
    'moonphase', 'Unnamed: 0.1', 'Unnamed: 0_y',
    'weathercode', 'temperature_2m_mean', 'preciptype',
    'sunrise', 'sunset', 'Unnamed: 0',
    'conditions'
]
df_khulna.drop(columns=features_to_drop, axis=1, inplace=True)

## 2) Split dataset
The dataset will be split into training, validation and testing dataset.

In [108]:
df_khulna['datetime'] = pd.to_datetime(df_khulna['datetime'], format='%d-%m-%Y')


In [109]:
df_train.head()

,datetime,tempmax_x,tempmin_x,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,uvindex,river_discharge,tempmax_y,tempmin_y,apparent_temperature_mean,precipitation_sum,rain_sum,precipitation_hours,windspeed_y,et0_fao_evapotranspiration
0,2015-01-01,28.9,17.5,21.2,28.8,17.5,21.2,16.5,76.7,0.9,...,6.0,71.44,21.7,18.3,22.0,7.0,7.0,13.0,10.8,0.82
1,2016-01-01,28.0,12.8,19.4,27.5,12.8,19.4,11.8,65.5,0.0,...,6.0,73.99,26.2,13.8,20.5,0.0,0.0,0.0,9.0,3.04
2,2017-01-01,26.7,14.5,19.8,26.7,14.5,19.8,14.6,74.6,0.0,...,6.0,74.34,25.7,15.0,20.9,0.0,0.0,0.0,9.2,2.90
3,2018-01-01,24.8,13.3,18.9,24.8,13.3,18.9,15.6,82.6,0.0,...,7.0,90.41,24.6,13.5,20.2,0.0,0.0,0.0,9.8,2.18
4,2019-01-01,27.0,10.2,17.4,26.7,10.2,17.4,9.8,65.6,0.0,...,7.0,77.89,24.9,12.1,17.8,0.0,0.0,0.0,9.0,3.16


In [110]:
# Training dataset
df_train = df_khulna[(df_khulna['datetime'] >= '01-01-2013') &
                     (df_khulna['datetime'] <= '31-12-2019')]
df_train.to_csv("train_set.csv", index=False)

# Split into features and target variable
X_train = df_train.drop(columns=['precip', 'datetime'])
y_train = df_train['precip']

<ipython-input-110-60b35c6081f6>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  (df_khulna['datetime'] <= '31-12-2019')]


In [112]:
# Validation dataset
df_val = df_khulna[(df_khulna['datetime'] >= '01-01-2020') &
                   (df_khulna['datetime'] <= '31-12-2021')]
df_val.to_csv("val_set.csv", index=False)

# Split into features and target variable
X_val = df_val.drop(columns=['precip', 'datetime'])
y_val = df_val['precip']

<ipython-input-112-a19154f39a26>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  (df_khulna['datetime'] <= '31-12-2021')]


In [113]:
# Testing dataset
df_test = df_khulna[(df_khulna['datetime'] >= '01-01-2022') &
                    (df_khulna['datetime'] <= '31-12-2023')]
df_test.to_csv("test_set.csv", index=False)

# Split into features and target variable
X_test = df_test.drop(columns=['precip', 'datetime'])
y_test = df_test['precip']

<ipython-input-113-f8d423c679b6>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  (df_khulna['datetime'] <= '31-12-2023')]


## 3) Train the model


3.1) Linear Regression

> Linear regression is used as a the baseline model, to gauge the performance of other models. The validation dataset is not used as there are not many hyperparameters to tune for this model.



In [13]:
# Initialize the Linear Regression model
model = LinearRegression()

# Train the model using the training data
model.fit(X_train, y_train)

# Test the model using the testing data
test_predictions = model.predict(X_test)

# Calculate evaluation metrics
test_mae = mean_absolute_error(y_test, test_predictions)
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, test_predictions)

print(f"Test Mean Squared Error (RMSE): {test_rmse}")
print(f"Test Mean Absolute Error (MAE): {test_mae}")
print(f"Test R-squared (R2): {test_r2}")

Test Mean Squared Error (RMSE): 17.415694178124138
Test Mean Absolute Error (MAE): 15.711907247969087
Test R-squared (R2): -1.787321602571995


3.2) LSTM

> LSTM model is considered in time series forecasting due to its ability to model sequences and capture temporal dependencies effectively. Some of the important hyperparaneters of LSTM include: Number of LSTM Units, Activation Functions, Dropout, Batch Size, Learning Rate, Number of Layers, Input Sequence Length & Optimizer. <br>

In [14]:
train_df = pd.read_csv('train_set.csv', index_col='datetime',parse_dates=True)
val_df = pd.read_csv('val_set.csv', index_col='datetime',parse_dates=True)
test_df = pd.read_csv('test_set.csv', index_col='datetime',parse_dates=True)

In [15]:
print(train_df.shape, val_df.shape, test_df.shape)

(1703, 29) (731, 29) (594, 29)


In [16]:
class DataWindow():
    def __init__(self, input_width, label_width, shift,
                 train_df=train_df, val_df=val_df, test_df=test_df,
                 label_columns=None):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def split_to_inputs_labels(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:,:,self.column_indices[name]] for name in self.label_columns],
                axis=-1
            )
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels


    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32
        )

        ds = ds.map(self.split_to_inputs_labels)
        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def sample_batch(self):
        result = getattr(self, '_sample_batch', None)
        if result is None:
            result = next(iter(self.train))
            self._sample_batch = result
        return result

In [17]:
class Baseline(Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs

        elif isinstance(self.label_index, list):
            tensors = []
            for index in self.label_index:
                result = inputs[:, :, index]
                result = result[:, :, tf.newaxis]
                tensors.append(result)
            return tf.concat(tensors, axis=-1)

        result = inputs[:, :, self.label_index]
        return result[:,:,tf.newaxis]

In [18]:
mo_single_step_window = DataWindow(input_width=1, label_width=1, shift=1, label_columns=['precip'])
mo_wide_window = DataWindow(input_width=14, label_width=14, shift=1, label_columns=['precip'])

In [19]:
column_indices = {name: i for i, name in enumerate(train_df.columns)}

In [20]:
print(column_indices['precip'])

9


In [21]:
mo_baseline_last = Baseline(label_index=[9])

mo_baseline_last.compile(loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])

mo_val_performance = {}
mo_performance = {}

mo_val_performance['Baseline - Last'] = mo_baseline_last.evaluate(mo_wide_window.val)
mo_performance['Baseline - Last'] = mo_baseline_last.evaluate(mo_wide_window.test, verbose=0)

23/23 [==============================] - 0s 2ms/step - loss: 263.2781 - mean_absolute_error: 8.8402


In [22]:
def compile_and_fit(model, window, patience=3, max_epochs=50):
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=patience,
                                   mode='min')

    model.compile(loss=MeanSquaredError(),
                  optimizer=Adam(),
                  metrics=[MeanAbsoluteError()])

    history = model.fit(window.train,
                       epochs=max_epochs,
                       validation_data=window.val,
                       callbacks=[early_stopping])

    return history

In [23]:
mo_dense = Sequential([
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=1)
])

history = compile_and_fit(mo_dense, mo_single_step_window)

mo_val_performance['Dense'] = mo_dense.evaluate(mo_single_step_window.val)
mo_performance['Dense'] = mo_dense.evaluate(mo_single_step_window.test, verbose=0)

Epoch 1/50
54/54 [==============================] - 2s 17ms/step - loss: 2483.0862 - mean_absolute_error: 30.8546 - val_loss: 241.0117 - val_mean_absolute_error: 11.1917
Epoch 2/50
54/54 [==============================] - 1s 15ms/step - loss: 316.0856 - mean_absolute_error: 10.9264 - val_loss: 179.2884 - val_mean_absolute_error: 8.9031
Epoch 3/50
54/54 [==============================] - 1s 16ms/step - loss: 243.9617 - mean_absolute_error: 8.5949 - val_loss: 163.8826 - val_mean_absolute_error: 8.2216
Epoch 4/50
54/54 [==============================] - 1s 8ms/step - loss: 239.2326 - mean_absolute_error: 8.4729 - val_loss: 151.5876 - val_mean_absolute_error: 8.6410
Epoch 5/50
54/54 [==============================] - 1s 13ms/step - loss: 236.2831 - mean_absolute_error: 8.4234 - val_loss: 158.8638 - val_mean_absolute_error: 9.3728
Epoch 6/50
54/54 [==============================] - 0s 6ms/step - loss: 237.1257 - mean_absolute_error: 8.6502 - val_loss: 151.9744 - val_mean_absolute_error: 8.1

In [24]:
mo_lstm_model = Sequential([
    LSTM(32, return_sequences=True),
    Dense(units = 1)
])

history = compile_and_fit(mo_lstm_model, mo_wide_window)

mo_val_performance = {}
mo_performance = {}

mo_val_performance['LSTM'] = mo_lstm_model.evaluate(mo_wide_window.val)
mo_performance['LSTM'] = mo_lstm_model.evaluate(mo_wide_window.test, verbose=0)

Epoch 1/50
53/53 [==============================] - 3s 16ms/step - loss: 304.0356 - mean_absolute_error: 7.0515 - val_loss: 185.8231 - val_mean_absolute_error: 6.9450
Epoch 2/50
53/53 [==============================] - 1s 10ms/step - loss: 283.5568 - mean_absolute_error: 7.1633 - val_loss: 173.0440 - val_mean_absolute_error: 6.9180
Epoch 3/50
53/53 [==============================] - 0s 9ms/step - loss: 272.6982 - mean_absolute_error: 7.4253 - val_loss: 167.1859 - val_mean_absolute_error: 6.9698
Epoch 4/50
53/53 [==============================] - 1s 9ms/step - loss: 267.1964 - mean_absolute_error: 7.5342 - val_loss: 163.6880 - val_mean_absolute_error: 6.9620
Epoch 5/50
53/53 [==============================] - 0s 9ms/step - loss: 262.6027 - mean_absolute_error: 7.5496 - val_loss: 160.9524 - val_mean_absolute_error: 6.9176
Epoch 6/50
53/53 [==============================] - 1s 10ms/step - loss: 258.5974 - mean_absolute_error: 7.5535 - val_loss: 159.0126 - val_mean_absolute_error: 6.8619
E

In [25]:
predicted_results = mo_baseline_last.predict(mo_wide_window.test)
predicted_array= predicted_results[0]

my_array = np.array(predicted_array)

# df_raw = pd.DataFrame(my_array)

# df = df_raw.rename(columns={0: "precip"})

df = pd.DataFrame(my_array, columns=["precip"])

df.head(14)

19/19 [==============================] - 0s 2ms/step


,precip
0,0.0
1,0.0
2,0.1
3,3.2
4,0.0
5,0.4
6,2.5
7,0.0
8,11.1
9,13.3


In [26]:
mo_lstm_model.save("lstm_khulna_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## 4) Discussions

'precipitation_sum', 'rain_sum', 'precipitation_hours', 'precipcover', 'cloudcover', 'humidity' has strongest positive correlation with 'precip'.

LSTM has lower MAE value but higher RMSE than Linear Regression. The lower MAE value of the LSTM suggests that, on average, it has smaller errors in prediction compared to Linear Regression. However, a high RMSE means that LSTM has larger errors (outliers).

LSTM models are more complex and capable of capturing intricate patterns, especially in time-series data, which might result in a lower MAE. Linear Regression assumes a linear r/s between features and target variable which may not be adequate to model complex non-linear r/s present in data.

To improve, we can study the outliers and consider more hyperparameters for LSTM.